In [13]:
#many to many
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import matplotlib.pyplot as plt

torch.manual_seed(1)

# Hyperparameters
epochs = 1000
lr = 0.1
input_size = 2
hidden_size = 2
output_size = 2
batch_size = 1
seq_len = 20
num_layers = 1

# random data
inputs = torch.randn(batch_size, seq_len, input_size)
targets = torch.randn(batch_size, seq_len, output_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model
# many to many LSTM

class LSTMmany(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMmany, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstmencoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.lstmdecoder = nn.LSTM(hidden_size * 2, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))
    
    def forward(self, input):
        _, h = self.lstmencoder(input, self.hidden)
        latent = h[-(1 + int(self.lstmencoder.bidirectional)):]
        #decode the latent vector to (batch_size, seq_len, output_size)
        #latent is shape of (batch_size, hidden_size * num_directions)
        output, _ = self.lstmdecoder(latent.repeat(1, seq_len, 1), self.hidden)
        
        return output
    
model = LSTMmany(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train
for epoch in range(epochs):
    model.zero_grad()
    model.hidden = model.init_hidden()
    input = torch.Tensor(inputs).to(device)
    target = torch.Tensor(targets).to(device)
    output = model(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch: %d, loss: %.4f' % (epoch, loss.item()))

Epoch: 0, loss: -0.7346
Epoch: 100, loss: -169.1972
Epoch: 200, loss: -334.0926
Epoch: 300, loss: -493.5266
Epoch: 400, loss: -650.4119
Epoch: 500, loss: -805.0861
Epoch: 600, loss: -957.2609
Epoch: 700, loss: -1110.6610
Epoch: 800, loss: -1262.4364
Epoch: 900, loss: -1413.7457


In [14]:
class ManyDataset(torch.utils.data.Dataset):

    def __init__(self, X, y, day, input_len=128):
        #the input data is a 1d array, indicate the minute of the day
        self.X = X
        self.y = y
        self.day = day
        self.input_len = input_len

    def __getitem__(self, index):
        #output previous self.input_len minutes data and target value
        #if the there is no enough data in the same day, pad with 0
        #y is the target value of future input_len minutes
        d = self.day[index]
        start = index - self.input_len
        if start < 0:
            start = 0
        if self.day[start] != d:
            while self.day[start] != d:
                start += 1
            #pad with 0 before start
        if index - start < self.input_len:
            x = torch.zeros(self.input_len, self.X.shape[1])
            x[self.input_len - index + start: self.input_len] = self.X[start: index].clone()
        else:
            x = self.X[start: index]

        #generate y
        end = index + self.input_len
        if end >= len(self.X):
            end = len(self.X) - 1
        if self.day[end] != d:
            while self.day[end] != d:
                end -= 1
        #pad with 0 after end
        if end - index < self.input_len:
            y = torch.zeros(self.input_len)
            y[0: end - index] = self.y[index: end].clone()
        else:
            y = self.y[index: end]
            

        return x, y
    def __len__(self):
        return len(self.X)

In [23]:
#test ManyDataset
num_samples = 1000
X = torch.randn(num_samples, 2)
y = torch.randn(num_samples)
days = torch.randint(1, 10, (num_samples,))
dataset = ManyDataset(X, y, days)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)
for i, data in enumerate(dataloader):
    x, y = data
    print(x.shape, y.shape)


torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size([4, 128, 2]) torch.Size([4, 128])
torch.Size

In [20]:
for i in dataloader:
    print(i)


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'ManyDataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'ManyDataset' on <module '__main__' (built-in)>


RuntimeError: DataLoader worker (pid(s) 9553) exited unexpectedly

In [27]:
X= torch.rand(100009, 40)
#split into 100 chunks
chunks = torch.split(X, len(X) // 100, dim=0)
for chunk in chunks:
    print(chunk.shape)

torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size([1000, 40])
torch.Size(

In [17]:
import torch
import torch.nn as nn

#seq2seq for time series prediction
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, dropout=self.dropout, batch_first=True)

    def forward(self, input):
        # input shape: (batch_size, seq_len, input_dim)
        # output shape: (batch_size, seq_len, hidden_dim)
        # hidden shape: (n_layers, batch_size, hidden_dim)
        # cell shape: (n_layers, batch_size, hidden_dim)
        output, (hidden, cell) = self.lstm(input)
        return output, hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, dropout):
        super(Decoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, n_layers, dropout=self.dropout, batch_first=True)

    def forward(self, input, hidden, cell):
        # input shape: (batch_size, 1, input_dim)
        # hidden shape: (n_layers, batch_size, hidden_dim)
        # cell shape: (n_layers, batch_size, hidden_dim)
        # output shape: (batch_size, 1, hidden_dim)
        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        # output shape: (batch_size, output_dim)
        output = output.squeeze(1)
        return output, hidden, cell
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.fc = nn.Linear(decoder.hidden_dim, decoder.output_dim)
        assert encoder.n_layers == decoder.n_layers, "Encoder and decoder must have equal number of layers!"
        
    def forward(self, input):
        # input shape: (batch_size, seq_len, input_dim)
        batch_size = input.shape[0]
        seq_len = input.shape[1]
        output_dim = self.decoder.output_dim
        outputs = torch.zeros(batch_size, seq_len, output_dim).to(self.device)
        encoder_output, hidden, cell = self.encoder(input)
        # decoder_input shape: (batch_size, 1, output_dim)
        # use encoder_output as the first input to the decoder
        decoder_input = encoder_output[:, 0, :].unsqueeze(1)
        for t in range(seq_len):
            decoder_output, hidden, cell = self.decoder(decoder_input, hidden, cell)
            outputs[:, t, :] = self.fc(decoder_output)
            decoder_input = decoder_output.unsqueeze(1)
        return outputs
    
#generate random data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#generate time series data
#X is of shape (batch_size, seq_len, input_dim)
#y is of shape (batch_size, seq_len, output_dim)
def generate_data(batch_size, seq_len, input_dim, output_dim):
    X = np.random.randn(batch_size, seq_len, input_dim)
    y = np.random.randn(batch_size, seq_len, output_dim)
    return X, y

#generate data
batch_size = 64
seq_len = 10
input_dim = 5
output_dim = 2
X, y = generate_data(batch_size, seq_len, input_dim, output_dim)
print(X.shape)
print(y.shape)

#train model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = Encoder(input_dim, hidden_dim=10, n_layers=2, dropout=0.5)
decoder = Decoder(output_dim, hidden_dim=10, n_layers=2, dropout=0.5)
model = Seq2Seq(encoder, decoder, device).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    input = torch.from_numpy(X).float().to(device)
    target = torch.from_numpy(y).float().to(device)
    output = model(input)
    print(output.shape)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("Epoch [{}/{}], loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))



(64, 10, 5)
(64, 10, 2)
torch.Size([64, 10, 2])
Epoch [1/100], loss: 1.1006
torch.Size([64, 10, 2])
Epoch [2/100], loss: 1.0983
torch.Size([64, 10, 2])
Epoch [3/100], loss: 1.0952
torch.Size([64, 10, 2])
Epoch [4/100], loss: 1.0938
torch.Size([64, 10, 2])
Epoch [5/100], loss: 1.0909
torch.Size([64, 10, 2])
Epoch [6/100], loss: 1.0888
torch.Size([64, 10, 2])
Epoch [7/100], loss: 1.0857
torch.Size([64, 10, 2])
Epoch [8/100], loss: 1.0839
torch.Size([64, 10, 2])
Epoch [9/100], loss: 1.0815
torch.Size([64, 10, 2])
Epoch [10/100], loss: 1.0800
torch.Size([64, 10, 2])
Epoch [11/100], loss: 1.0774
torch.Size([64, 10, 2])
Epoch [12/100], loss: 1.0753
torch.Size([64, 10, 2])
Epoch [13/100], loss: 1.0731
torch.Size([64, 10, 2])
Epoch [14/100], loss: 1.0722
torch.Size([64, 10, 2])
Epoch [15/100], loss: 1.0698
torch.Size([64, 10, 2])
Epoch [16/100], loss: 1.0680
torch.Size([64, 10, 2])
Epoch [17/100], loss: 1.0663
torch.Size([64, 10, 2])
Epoch [18/100], loss: 1.0644
torch.Size([64, 10, 2])
Epoch [